In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [3]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/02/26 13:22:43] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/26 13:22:43] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/26 13:22:43] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [8]:
gui = GUI('data/input/0.jpg')
gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
# gui.load_detection_result()
gui.show_detection_result()

[2022/02/25 21:44:33] root DEBUG: dt_boxes num : 23, elapse : 0.14209628105163574
[2022/02/25 21:44:33] root DEBUG: cls num  : 23, elapse : 0.08286499977111816
[2022/02/25 21:44:33] root DEBUG: rec_res num  : 23, elapse : 0.1570730209350586
[Text Detection Completed in 0.434 s] Input: data/input/0.jpg Output: data/output/ocr/0.json
[Compo Detection Completed in 0.386 s] Input: data/input/0.jpg Output: data/output/ip/0.json
[Merge Completed] Input: data/input/0.jpg Output: data/output/merge/0.jpg


In [6]:
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.resize(gui.detection_resize_width, gui.detection_resize_height)        
    operations.append(op)
#     op.show_target_ele()

In [9]:
gui.match_elements(op.target_element_img, resnet_model=resnet_model)

In [10]:
operations[0].show_target_ele()

In [9]:
# ***************************
# *** Streaming Detection ***
# ***************************

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while(1):
    ret, frame = cap.read()
    
    cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.imwrite('data/output/temp.jpg', frame)
        gui = GUI('data/output/temp.jpg')
        gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
        cv2.imshow('result', gui.det_result_imgs['merge'])
        print('===================\n')
cap.release()
cv2.destroyAllWindows()

[2022/02/26 13:25:22] root DEBUG: dt_boxes num : 0, elapse : 0.14470863342285156
[2022/02/26 13:25:22] root DEBUG: cls num  : 0, elapse : 0
[2022/02/26 13:25:22] root DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[Text Detection Completed in 0.161 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 1.063 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg



In [6]:
cap.release()